# Word Normalization

### José Pablo Kiesling Lange

In [1]:
import re
from collections import Counter

from tokenizers import Tokenizer, models, trainers, pre_tokenizers

from nltk.stem import SnowballStemmer

from Levenshtein import distance

In [2]:
corpus_path = "data/escher_comments.txt"

In [3]:
with open(corpus_path, "r", encoding="utf-8") as file:
    corpus = file.readlines()

In [4]:
corpus = [line.strip() for line in corpus if line.strip()]

## Estandarización

Para ver la efectivdad de la estandarización, se mostrará las 10 palabras más frecuentes del corpus antes y después de la estandarización. El objetivo es ver si hay modificación en la cantidad de dichas palabras o si una nueva palabra aparece con frecuencia.

In [5]:
def get_most_common_words(corpus, n=10):
    words = [word for line in corpus for word in line.split()]
    most_common = Counter(words).most_common(n)
    return most_common

In [6]:
most_common_words = get_most_common_words(corpus, n=10)
most_common_words

[('que', 45),
 ('de', 39),
 ('la', 32),
 ('un', 27),
 ('una', 26),
 ('el', 17),
 ('en', 16),
 ('esfera', 15),
 ('reflejo', 15),
 ('y', 14)]

#### Case Folding

Se hace una función que dado un corpus revisa si posee la misma palabra en mayúscula y minúscula.

In [7]:
def get_upper_lower_words(corpus):
    words = set(word for line in corpus for word in line.split())
    upper_lower_words = set()
    for word in words:
        if word.lower() in words and word.upper() in words:
            upper_lower_words.add(word)
    return upper_lower_words

In [8]:
upper_lower_words = get_upper_lower_words(corpus)
upper_lower_words

{'A', 'a'}

Dado que sí hay por lo menos una palabra que aparece en ambas formas, se procede a hacer un case folding (pasar todas las palabras a minúscula).

In [9]:
def case_folding(corpus):
    return [line.lower() for line in corpus]

#### Remove Punctuation

Dado que en las instrucciones para generar el corpus se pidió mínimo 2 oraciones, se asume que mínimo una de las oraciones tiene puntuación. Por lo tanto, se procede a eliminar la puntuación del corpus.

Esto se hace para evitar que palabras como `la,` o `la.` se cuenten como palabras diferentes a `la`.

In [10]:
def remove_punctuation(corpus):
    return [re.sub(r'[^\w\s]', '', line) for line in corpus]

In [11]:
def standarizate(corpus):
    corpus = case_folding(corpus)
    corpus = remove_punctuation(corpus)
    return corpus

In [12]:
corpus = standarizate(corpus)

### Corpus ya estandarizado

In [13]:
most_common_words = get_most_common_words(corpus, n=10)
most_common_words

[('que', 45),
 ('de', 39),
 ('la', 34),
 ('un', 30),
 ('una', 28),
 ('esfera', 18),
 ('en', 18),
 ('el', 17),
 ('reflejo', 16),
 ('se', 15)]

Como se puede apreciar:
- la palabra `la` se incrementó en `2`ocurrencias
- la palabra `un` se incrementó en `3` ocurrencias
- la palabra `una` se incrementó en `3`ocurrencias
- la palabra `esfera` se incrementó en `3` ocurrencias y subió al `6to` lugar
- la palabra `en` se incrementó en `2` ocurrencias y subió al `7mo` lugar
- la palabra `reflejo` se incrementó en `1` ocurrencia
- la palabra `se` **ingresó** al top 10

Por lo tanto, la estandarización ha tenido un efecto positivo en la, ya que ha incrementado la cantidad de ocurrencias de palabras comunes y ha eliminado las variaciones causadas por puntuación.

Algo a recalcar es la ausencia de la estandarización de palabras con o sin tilde. Esto es porque se entrenará el modelo `BPE` en español.

## BPE

La implementación se hará en base a la implementación de `LangformersBlog`

https://blog.langformers.com/bpe-tokenizer-explained/

En la instanciación del `BpeTrainer` se especifica el tamaño del vocabulario y los tokens especiales. Para definir el tamaño del vocabulario, se debe considerar el tamaño del corpus y la cantidad de palabras que se espera que aparezcan en el corpus. Por lo tanto, se hacen métodos para calcular el tamaño del vocabulario y la cantidad de palabras que se espera que aparezcan en el corpus.

In [14]:
def get_vocab_size(corpus):
    return len(set(word for line in corpus for word in line.split()))

In [15]:
vocab_size = get_vocab_size(corpus)
print(f"Vocabulario estimado por tokens únicos: {vocab_size}")

Vocabulario estimado por tokens únicos: 229


Dado que el corpus es pequeño, se define un tamaño de vocabulario de `300` y los tokens especiales son `[PAD]` y `[UNK]`.

In [16]:
tokenizer = Tokenizer(models.BPE())

In [17]:
trainer = trainers.BpeTrainer(
    vocab_size=300,
    special_tokens=["[PAD]", "[UNK]"]
)

El token `[PAD]` se utiliza para rellenar secuencias de longitud variable, mientras que el token `[UNK]` se utiliza para representar palabras desconocidas o fuera del vocabulario.

Ejemplo:
Si el corpus contiene las palabras `hola`, `mundo`, y `adiós`, el vocabulario podría ser:

```
{
    "[PAD]": 0,
    "[UNK]": 1,
    "hola": 2,
    "mundo": 3,
    "adiós": 4
}
```

Entonces si viniera el texto `hello mundo adiós` y otro texto `hola mundo`, el tokenizador lo convertiría a
1. [1, 3, 4] (donde `1` es el token `[UNK]` para `hello`)
2. [2, 3, 0] (donde `0` es el token `[PAD]` para la secuencia de longitud variable)

Es por ello que el uso de ambos tokens es importante para manejar secuencias de longitud variable y palabras desconocidas. En el caso de `PAD` es útil para rellenar secuencias más cortas, mientras que `UNK` es esencial para manejar palabras que no están en el vocabulario.

Sin embargo, en el caso de `PAD`, normalmente no aparece en el vocabulario final porque se añade a las secuencias al momento de batching no durante el entrenamiento del tokenizador.


In [18]:
tokenizer.train_from_iterator(corpus, trainer)

In [19]:
print("Vocabulary:", tokenizer.get_vocab())
tokenizer.save("tiny_tokenizer.json")

Vocabulary: {'ombre ': 81, 'sí mismo ': 173, 'v': 23, 'ti': 65, 'bros ': 197, 'que es ': 191, 'tiene ': 104, 'so': 140, 'mano ': 171, 'ver ': 204, 'jo ': 220, 'u': 22, 'li': 155, 'qu': 41, 'ta': 70, 'pi': 226, 'de la ': 143, 'or ': 132, 'pu': 157, 'sen': 296, 'una mano ': 194, 'con': 255, 'do ': 61, 'ebles ': 217, 'de': 98, 'oca ': 225, 'bu': 276, 'da la ': 215, 'ar ': 167, 'espe': 236, 'pl': 293, 'sí m': 151, 'alrede': 198, 'reflexión ': 193, '[UNK]': 1, 'co ': 279, 'es un ': 245, 'no ': 108, 'refle': 49, 'x': 24, 'parec': 186, 'a': 3, 'alreded': 209, 'ad': 112, 'mos ': 184, 't': 21, 'en un ': 243, 'ene ': 102, 's ': 90, 'on': 223, 'alg': 248, 'entorno ': 271, 'lu': 156, 'más ': 139, 'r': 19, 'arec': 175, 'se v': 110, 'ando ': 161, 'bles ': 212, ' lo que ': 273, 'b': 4, 'libros ': 206, 'bita': 134, 'n': 15, 'po': 227, 'en ': 58, 'cu': 176, 'ó': 31, 'y': 25, 'refleja ': 162, 'e ': 33, 'propi': 261, 'fer': 67, 'co': 106, 'en': 36, 'pro': 165, 'mente ': 207, 'su ': 93, 'está ': 149, 'p':

## Comparación con WordPiece y SentencePiece

### WordPiece

In [20]:
wordPiece_tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
wordPiece_tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
wordPiece_trainer = trainers.WordPieceTrainer(vocab_size=300, special_tokens=["[PAD]", "[UNK]"])
wordPiece_tokenizer.train_from_iterator(corpus, wordPiece_trainer)

### SentencePiece

In [21]:
sentencePiece_tokenizer = Tokenizer(models.BPE())
sentencePiece_tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
sentencePiece_trainer = trainers.BpeTrainer(vocab_size=300, special_tokens=["[PAD]", "[UNK]"])
sentencePiece_tokenizer.train_from_iterator(corpus, sentencePiece_trainer)

In [22]:
sample = corpus[0][:60]

In [23]:
print("Sample text:", sample)
print("BPE Tokenization:", tokenizer.encode(sample).tokens)
print("WordPiece Tokenization:", wordPiece_tokenizer.encode(sample).tokens)
print("SentencePiece Tokenization:", sentencePiece_tokenizer.encode(sample).tokens)

Sample text: veo a un hombre explorando su propia percepción al sostener 
BPE Tokenization: ['veo a ', 'un hombre ', 'e', 'x', 'pl', 'or', 'ando ', 'su ', 'propi', 'a ', 'per', 'ce', 'pción ', 'al ', 'sosten', 'er', ' ']
WordPiece Tokenization: ['veo', 'a', 'un', 'hombre', 'e', '##x', '##p', '##lo', '##ra', '##n', '##do', 'su', 'propi', '##a', 'per', '##ce', '##pción', 'al', 'sosten', '##er']
SentencePiece Tokenization: ['veo', 'a', 'un', 'hombre', 'e', 'x', 'p', 'lo', 'r', 'ando', 'su', 'propia', 'percepción', 'al', 'sosten', 'er']


Como se puede apreciar, en `BPE` aprende tokens frecuentes como secuencias `('veo a ', 'un hombre ')`. A veces no separa en palabras individuales, sino en fragmentos largos o combinaciones si son frecuentes. Puede incluir espacios dentro de los tokens.

Por otro lado, `WordPiece` tiene una peculiaridad y es el uso de `##`para indicar que la subpalabra es parte de una palabra más larga. Además, fragmenta más las palabras conservando la semántica.

Finalmente, `SentencePiece` es similar a `BPE` (lo cual hace sentido ya que este modelo usa este algoritmo), pero no utiliza espacios como separadores. Tampoco usa `##`pero puede segmentar de forma más limpias las palabras, ya que no depende de espacios.

Por lo que en conclusión, `BPE` y `SentencePiece` son más flexibles en la segmentación de palabras, mientras que `WordPiece` es más estricto y conservador, lo que puede ser útil para modelos que requieren una segmentación más precisa de las palabras.


## Snowball Stemmer

In [24]:
stemmer = SnowballStemmer("spanish")

In [25]:
sample_stemmed = ""

In [26]:
words = sample.split()

In [27]:
for token in words:
    sample_stemmed += stemmer.stem(token) + " "
    print(f"Original: {token}, Stemmed: {stemmer.stem(token)}")
    
print("\nSample stemmed:", sample_stemmed.strip())

Original: veo, Stemmed: veo
Original: a, Stemmed: a
Original: un, Stemmed: un
Original: hombre, Stemmed: hombr
Original: explorando, Stemmed: explor
Original: su, Stemmed: su
Original: propia, Stemmed: propi
Original: percepción, Stemmed: percepcion
Original: al, Stemmed: al
Original: sostener, Stemmed: sosten

Sample stemmed: veo a un hombr explor su propi percepcion al sosten


Al hacer el stemming, se puede apreciar que verbos mantienen su raíz para que se pueda entender su lema. Por ejemplo, la subpalabra `explorando` se convierte en `explor`, la cual puede tener diferentes conjugaciones, y se sigue entendiendo como `explorar`.
Sin embargo, en el caso de sustantivos, se pierde la relación semántica original. Por ejemplo, la subpalabra `hombr` se puede entender como `hombre` u `hombro`. Esto se debe a que el algoritmo de stemming utilizado no tiene en cuenta el contexto de la palabra, por lo que puede generar raíces ambiguas.

Además, la subpalabra `percepción`se le remueve la tilde y se convierte en `percepcion`. Lo cual puede ser problemático, ya que en español la tilde es importante para la correcta pronunciación y significado de las palabras. Por lo tanto, al hacer el stemming, se pierde información importante sobre la palabra.

Por lo tanto, el stemming es útil para reducir las palabras a su raíz, pero puede generar raíces ambiguas y perder información importante sobre la palabra. En este caso, se recomienda utilizar lematización en lugar de stemming para preservar la relación semántica original de las palabras.

## Levenshtein